## 3.4 CLIP+DINOv2

### CLIP-based

In [1]:
import torch
from PIL import Image
from transformers import AutoProcessor, CLIPModel
import torch.nn as nn

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
processor = AutoProcessor.from_pretrained("/sun/home_models/clip-vit-large-patch14")
model = CLIPModel.from_pretrained("/sun/home_models/clip-vit-large-patch14").to(device)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [5]:
#Extract features from image1
image1 = Image.open('/sun/home_datasets/iharmony4_t_256/'
                    'Hday2night/composite_images/d16761-82_1_1.png')
with torch.no_grad():
    inputs1 = processor(images=image1, return_tensors="pt").to(device)
    image_features1 = model.get_image_features(**inputs1)

#Extract features from image2
image2 = Image.open('/sun/home_datasets/iharmony4_t_256/'
                    'Hday2night/real_images/d16761-82.png')
with torch.no_grad():
    inputs2 = processor(images=image2, return_tensors="pt").to(device)
    image_features2 = model.get_image_features(**inputs2)

In [4]:
#Compute their cosine similarity and convert it into a score between 0 and 1
cos = nn.CosineSimilarity(dim=0)
sim = cos(image_features1[0],image_features2[0]).item()
sim = (sim+1)/2
print('Similarity:', sim)

Similarity: 0.9802383184432983


### 真图的前后景的相关性和合成图的前后景的相关性

In [3]:
#Extract features from image1
image1 = Image.open('/sun/home_datasets/iharmony4_t_256/'
                    'Hday2night/composite_images/d16761-82_1_1.png')
with torch.no_grad():
    inputs1 = processor(images=image1, return_tensors="pt").to(device)
    image_features1 = model.get_image_features(**inputs1)

In [5]:
image_features1.shape

torch.Size([1, 768])

In [ ]:
#Extract features from image2
image2 = Image.open('/sun/home_datasets/iharmony4_t_256/'
                    'Hday2night/real_images/d16761-82.png')
with torch.no_grad():
    inputs2 = processor(images=image2, return_tensors="pt").to(device)
    image_features2 = model.get_image_features(**inputs2)

In [4]:
#Compute their cosine similarity and convert it into a score between 0 and 1
cos = nn.CosineSimilarity(dim=0)
sim = cos(image_features1[0],image_features2[0]).item()
sim = (sim+1)/2
print('Similarity:', sim)

Similarity: 0.9802383184432983


### DINOV2

In [9]:
import torch
from PIL import Image
from transformers import AutoProcessor, CLIPModel, AutoImageProcessor, AutoModel
# import faiss
import os
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

#Load DINOv2 model and processor
processor_dino = AutoImageProcessor.from_pretrained('/sun/home_models/dinov2/dinov2_vitl14_pretrain.pth')
model_dino = AutoModel.from_pretrained('/sun/home_models/dinov2/dinov2_vitl14_pretrain.pth').to(device)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 64: invalid start byte

In [5]:
#Extract features from image1
image1 = Image.open('/sun/home_datasets/iharmony4_t_256/'
                    'Hday2night/composite_images/d16761-82_1_1.png')
with torch.no_grad():
    inputs1 = processor_dino(images=image1, return_tensors="pt").to(device)
    image_features1 = model_dino(**inputs1)

#Extract features from image2
image2 = Image.open('/sun/home_datasets/iharmony4_t_256/'
                    'Hday2night/real_images/d16761-82.png')
with torch.no_grad():
    inputs2 = processor_dino(images=image2, return_tensors="pt").to(device)
    image_features2 = model_dino(**inputs2)

In [ ]:
 #Retrieve all filenames
 images = []
 for root, dirs, files in os.walk('./val2017/'):
    for file in files:
        if file.endswith('jpg'):
            images.append(root + '/'+ file)


 #Define a function that normalizes embeddings and add them to the index
 def add_vector_to_index(embedding, index):
    #convert embedding to numpy
    vector = embedding.detach().cpu().numpy()
    #Convert to float32 numpy
    vector = np.float32(vector)
    #Normalize vector: important to avoid wrong results when searching
    faiss.normalize_L2(vector)
    #Add to index
    index.add(vector)

 def extract_features_clip(image):
    with torch.no_grad():
        inputs = processor_clip(images=image, return_tensors="pt").to(device)
        image_features = model_clip.get_image_features(**inputs)
        return image_features

 def extract_features_dino(image):
    with torch.no_grad():
        inputs = processor_dino(images=image, return_tensors="pt").to(device)
        outputs = model_dino(**inputs)
        image_features = outputs.last_hidden_state
        return image_features.mean(dim=1)

 #Create 2 indexes.
 index_clip = faiss.IndexFlatL2(512)
 index_dino = faiss.IndexFlatL2(768)

 #Iterate over the dataset to extract features X2 and store features in indexes
 for image_path in images:
    img = Image.open(image_path).convert('RGB')
    clip_features = extract_features_clip(img)
    add_vector_to_index(clip_features,index_clip)
    dino_features = extract_features_dino(img)
    add_vector_to_index(dino_features,index_dino)

 #store the indexes locally
 faiss.write_index(index_clip,"clip.index")
 faiss.write_index(index_dino,"dino.index")